<a href="https://colab.research.google.com/github/joshua-stock/fl-official-statistics/blob/main/med-insurance/fl-tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hands-on introduction to FL in OS

# Federated Insurance


## Tensorflow Federated Tutorials

**Getting started**

1. [Federated Learning for image classification](https://www.tensorflow.org/federated/tutorials/federated_learning_for_image_classification)
1. [Federated Learning for Text Generation](https://www.tensorflow.org/federated/tutorials/federated_learning_for_text_generation)
1. [Tuning recommended aggregations for learning](https://www.tensorflow.org/federated/tutorials/tuning_recommended_aggregators)
1. [Federated Reconstruction for Matrix Factorization](https://www.tensorflow.org/federated/tutorials/federated_reconstruction_for_matrix_factorization)

**... and  [more](https://www.tensorflow.org/federated/tutorials/tutorials_overview)**



## Setup

In [14]:
# Setup colab if needed

import os


try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

print("COLAB? {}".format(IN_COLAB))

if IN_COLAB:

    # rm repo from gdrive
    if os.path.exists("fl-official-statistics"):
      %rm -r fl-official-statistics

    # clone
    !git clone https://github.com/joshua-stock/fl-official-statistics
    %cd fl-official-statistics

    # pull (the currenct version of the repo)
    !git pull

    !pip install -q tensorflow-federated==0.56.0
    # or possibly !pip install -r requirements.txt

    os.chdir("med-insurance")
    
# suppress tf debug logging
# =========================
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

#0 = all messages are logged (default behavior)
#1 = INFO messages are not printed
#2 = INFO and WARNING messages are not printed
#3 = INFO, WARNING, and ERROR messages are not printed

# S. https://stackoverflow.com/questions/35911252/disable-tensorflow-debugging-information

COLAB? False


## Minimal example with wrappers

In [7]:


import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Data and inputs
client_var = 'region'
target = 'charges'
df_raw  = pd.read_csv('data/insurance.csv')

# preprocessing
df = df_raw.copy()
df[['sex', 'smoker']] = OrdinalEncoder().fit_transform(
    df[['sex', 'smoker']].astype('category'))
df[['age', 'bmi', 'children']] = MinMaxScaler(
  ).fit_transform(df[['age', 'bmi', 'children']])

clients = df[client_var].unique()

df.head()

,age,sex,bmi,children,smoker,region,charges
0,0.021739,0.0,0.321227,0.0,1.0,southwest,16884.92400
1,0.000000,1.0,0.479150,0.2,0.0,southeast,1725.55230
2,0.217391,1.0,0.458434,0.6,0.0,southeast,4449.46200
3,0.326087,1.0,0.181464,0.0,0.0,northwest,21984.47061
4,0.304348,1.0,0.347592,0.0,0.0,northwest,3866.85520


In [3]:
from FLutils import (
    create_keras_model,    # construct a deep neural network (keras)
    model_fn,              # convert keras model to tff.learning.models
    prep_fed_train,        # convert training data to tensors for learning with tensorflow
    prep_fed_test,         # convert test data to tensors for testing with tensorflow (other format than training data)
    train_fed              # train a keras model federated with distributed data
    )

In [4]:
def keras_blueprint(compile = False, nfeatures = None):
    if nfeatures == None: nfeatures = len(df.columns) - 2

    return create_keras_model(
        nfeatures = nfeatures,
        units = [40, 40, 20],
        activations = ['relu'] * 3,
        compile = compile)

In [5]:
train_data, test_data = train_test_split(
      df, test_size = 0.2, random_state = 42)

In [6]:
train_data_fed = []
test_data_fed = []



for client in clients:
  df_client = train_data[train_data[client_var] == client]
  df_client_train = df_client
  train_data_fed.append(
      prep_fed_train(
        df_client_train.loc[:,~ df_client_train.columns.isin([target, client_var])],
        df_client_train[target]
  ))
train_data_fed

2023-09-01 16:59:41.436164: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-01 16:59:41.645116: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-01 16:59:41.645214: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-01 16:59:41.650795: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-01 16:59:41.650906: I tensorflow/compile

[<_TensorSliceDataset element_spec=(TensorSpec(shape=(5,), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.float64, name=None))>,
 <_TensorSliceDataset element_spec=(TensorSpec(shape=(5,), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.float64, name=None))>,
 <_TensorSliceDataset element_spec=(TensorSpec(shape=(5,), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.float64, name=None))>,
 <_TensorSliceDataset element_spec=(TensorSpec(shape=(5,), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.float64, name=None))>]

In [57]:
# Training
result =  train_fed(
        model = model_fn(
            keras_creator = keras_blueprint,
            loss = tf.losses.MeanSquaredError()
        ),

        train_data = train_data_fed,

        NUM_ROUNDS = 5,
        NUM_EPOCHS = 5,

        client_optimizer = lambda: tf.optimizers.Adam(learning_rate = .05),
        server_optimizer = lambda: tf.optimizers.Adam(learning_rate = .05),

        BATCH_SIZE = 128,
        SHUFFLE_BUFFER = 20,
        PREFETCH_BUFFER = 5,
        SEED = 42,
        verbose = False
    )

In [59]:
# Evaluation
weights = result['process'].get_model_weights(result['state'])

model = keras_blueprint(compile = True)
weights.assign_weights_to(model)

perf_test = model.evaluate(test_data.loc[:,~test_data.columns.isin([target, client_var])].head(), test_data[target].head())
dict(zip(model.metrics_names, perf_test))


1/1 [==============================] - 0s 188ms/step - loss: 435178688.0000 - mae: 17242.2754 - mean_squared_error: 435178688.0000 - r2_score: -2.1475


{'loss': 435178688.0,
 'mae': 17242.275390625,
 'mean_squared_error': 435178688.0,
 'r2_score': -2.147523880004883}

## Minimal example no wrapper

In [21]:
(create_keras_model().input.shape[0], create_keras_model().input.shape[1])

(None, 5)

In [26]:
[x for x in create_keras_model().input.shape]

[None, 5]

In [19]:
# imports
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import InputLayer, Dense
import tensorflow_federated as tff
from sklearn.metrics import r2_score

# parameter

SEED = 1234

# a. data
client_var = 'region'
target     = 'charges'

# b. model architecture
def create_keras_model():
  return tf.keras.models.Sequential([
      InputLayer(input_shape=[5]),
      Dense(40, activation='relu'),
      Dense(40, activation='relu'),
      Dense(20, activation='relu'),
      Dense(1)
  ])

# c. training
NUM_ROUNDS = 5 # communication
NUM_EPOCHS = 5 # local client rounds
BATCH_SIZE = 128
SHUFFLE_BUFFER = 20
PREFETCH_BUFFER = 5

# ingest data
df_raw  = pd.read_csv('data/insurance.csv')

# preprocessing
df = df_raw.copy()

df[['sex', 'smoker']] = OrdinalEncoder(
  ).fit_transform(df[['sex', 'smoker']].astype('category'))
df[['age', 'bmi', 'children']] = MinMaxScaler(
  ).fit_transform(df[['age', 'bmi', 'children']])

clients = df[client_var].unique()

# train-test-split
# optional: cv
train_data, test_data = train_test_split(df, test_size = 0.2, random_state = SEED)

# distribute data to clients and convert to tensor
train_data_fed = []
for client in clients:

  X_client = train_data.loc[train_data[client_var] == client,  ~df.columns.isin([target, client_var])]
  y_client = train_data.loc[train_data[client_var] == client,  target]

  tensor_client = tf.data.Dataset.from_tensor_slices((
    tf.convert_to_tensor(X_client),
    tf.convert_to_tensor(y_client)))

  train_data_fed.append(tensor_client)


# convert model for FL
def model_fn():
  keras_model = create_keras_model()
  return tff.learning.models.from_keras_model(
      keras_model,
      input_spec = (
        tf.TensorSpec((
          keras_model.input.shape[0], 
          keras_model.input.shape[1]
          ), dtype = tf.float64),
        tf.TensorSpec((None,), dtype = tf.float64)
      ), loss = tf.keras.losses.MeanAbsoluteError(),
      metrics =  [
        tf.keras.metrics.MeanAbsoluteError()
        , tf.keras.metrics.MeanSquaredError()
        ]
  )

# Training
train_data_fed_proc = [
  data.
    repeat(NUM_EPOCHS).
    shuffle(SHUFFLE_BUFFER, seed = SEED).
    batch(BATCH_SIZE).
    prefetch(PREFETCH_BUFFER)
  for data in train_data_fed]

process = tff.learning.algorithms.build_weighted_fed_avg(
  model_fn,
	client_optimizer_fn = lambda: tf.optimizers.Adam(learning_rate = .05),
	server_optimizer_fn = lambda: tf.optimizers.Adam(learning_rate = .05))

state = process.initialize()

hist = []

for round in range(NUM_ROUNDS):
  state, perf = process.next(state, train_data_fed_proc)
  hist.append(dict(perf['client_work']['train'].items()))

print("== Training Performance ==")
print(dict(perf['client_work']['train'].items()))

# optional: eval
# ...

# test
model = create_keras_model()
model.compile(
    loss = 'mean_squared_error',
    metrics = ['mean_squared_error',"mae", r2_score],
    run_eagerly = True)

weights = process.get_model_weights(state)
weights.assign_weights_to(model)

perf_test = model.evaluate(
    test_data.loc[:,~df.columns.isin([target, client_var])],
    test_data[target],
    verbose = 0)

print("== Testing Performance ==")
print(dict(zip(model.metrics_names, perf_test)))


2023-09-01 17:08:51.493775: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-01 17:08:51.493868: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-09-01 17:08:51.494000: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-09-01 17:08:51.494717: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-01 17:08:51.494789: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-01 17:08:51.494837: I tensor

== Training Performance ==
{'mean_absolute_error': 11734.731, 'mean_squared_error': 278550430.0, 'loss': 11895.288, 'num_examples': 5350, 'num_batches': 44}
== Testing Performance ==
{'loss': 312832640.0, 'mean_squared_error': 312832640.0, 'mae': 13012.8046875, 'r2_score': -1.2668614387512207}
